In [46]:
import numpy as np
import pandas as pd

import re
import nltk as nlp 

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB


In [47]:
url='http://yustiks.ru/dataset/SPAM_text.csv'
data=pd.read_csv(url) 
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [48]:
data['Category'] = data.Category.apply(lambda row: 1 if row == 'spam' else 0)
data.head()

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [49]:
# translate the text into separate words
nlp_data = str(data.loc[2, 'Message'])

nlp_data = re.sub("[^a-zA-Z]"," ", nlp_data)

nlp_data = nlp_data.lower()
print(nlp_data)

free entry in   a wkly comp to win fa cup final tkts   st may       text fa to       to receive entry question std txt rate t c s apply            over   s


In [50]:
#import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context
# nlp.download()
# when you run nltk.download(), a window will appear where you can choose which packages to download

# remove the spaces
nlp.download('punkt')
nlp_data = nlp.word_tokenize(nlp_data)
print(nlp_data)

['free', 'entry', 'in', 'a', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkts', 'st', 'may', 'text', 'fa', 'to', 'to', 'receive', 'entry', 'question', 'std', 'txt', 'rate', 't', 'c', 's', 'apply', 'over', 's']


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>


In [51]:
# looking for the roots of the words
nlp.download('wordnet')
lemma = nlp.WordNetLemmatizer()
nlp_data = [lemma.lemmatize(word) for word in nlp_data]
# and add them to our list
nlp_data = "".join(nlp_data)

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>


In [52]:
description_list = []
for description in data["Message"]:
    description = re.sub("[^a-zA-Z]"," ",description).lower()
    description = nlp.word_tokenize(description)
    lemma = nlp.WordNetLemmatizer()
    description = [ lemma.lemmatize(word) for word in description]
    description = " ".join(description)
    description_list.append(description)
print(len(description_list))

5572


Creating a bag-of-words, to do this, select 3000 maximum encountered words

In [53]:
max_features = 3000
count_vectorizer = CountVectorizer(max_features = max_features, stop_words = "english")
sparce_matrix = count_vectorizer.fit_transform(description_list).toarray()
print(max_features,count_vectorizer.get_feature_names())

3000 ['aah', 'aathi', 'abi', 'ability', 'abiola', 'abj', 'able', 'absolutly', 'abt', 'abta', 'aburo', 'ac', 'academic', 'acc', 'accept', 'access', 'accident', 'accidentally', 'accordingly', 'account', 'ache', 'acl', 'aco', 'acted', 'acting', 'action', 'activate', 'active', 'activity', 'actor', 'actual', 'actually', 'ad', 'adam', 'add', 'addamsfa', 'added', 'addicted', 'addie', 'address', 'admin', 'administrator', 'admirer', 'admit', 'adore', 'adoring', 'adult', 'advance', 'adventure', 'advice', 'advise', 'ae', 'aeronautics', 'aeroplane', 'affair', 'affection', 'afraid', 'aft', 'afternoon', 'aftr', 'ag', 'agalla', 'age', 'agent', 'ago', 'agree', 'ah', 'aha', 'ahead', 'ahmad', 'aid', 'aight', 'ain', 'aint', 'air', 'airport', 'airtel', 'aiya', 'aiyah', 'aiyar', 'aiyo', 'aj', 'aka', 'al', 'alaipayuthe', 'album', 'alcohol', 'alert', 'alex', 'alfie', 'algarve', 'ali', 'alive', 'allah', 'allow', 'allowed', 'alright', 'alrite', 'alwys', 'amazing', 'american', 'amp', 'amt', 'amused', 'amy', 'an

In [54]:
# the most common word in all messages is 'crazy'
list_names = count_vectorizer.get_feature_names()
for i in range(len(list_names)):
    if list_names[i] == 'crazy':
        print(i)

592


In [55]:
# convert the source data to bag-of-words format
sparce_matrix[0, 592]

1

In [56]:
print(len(sparce_matrix[0,: ]))

3000


In [57]:
# divide the data into training and test data
y = data.iloc[:,0].values
x = sparce_matrix

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

In [58]:
# a naive Bayesian classifier
nb = GaussianNB()
nb.fit(x_train,y_train)
print("the accuracy of our model: {}".format(nb.score(x_test,y_test)))

the accuracy of our model: 0.8753363228699551


In [59]:
print(classification_report(y_test, nb.predict(x_test)))

              precision    recall  f1-score   support

           0       0.98      0.87      0.92       966
           1       0.52      0.89      0.66       149

    accuracy                           0.88      1115
   macro avg       0.75      0.88      0.79      1115
weighted avg       0.92      0.88      0.89      1115



In [60]:
# a logistic regression
lr = LogisticRegression(max_iter = 200)
lr.fit(x_train,y_train)
print("our accuracy is: {}".format(lr.score(x_test,y_test)))

our accuracy is: 0.9820627802690582


In [61]:
print(classification_report(y_test, lr.predict(x_test)))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       1.00      0.87      0.93       149

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [62]:
# k-NN classifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train,y_train)
#print('Prediction: {}'.format(prediction))
print('With KNN (K=3) accuracy is: ',knn.score(x_test,y_test))

With KNN (K=3) accuracy is:  0.9399103139013453


In [63]:
print(classification_report(y_test, knn.predict(x_test)))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       966
           1       1.00      0.55      0.71       149

    accuracy                           0.94      1115
   macro avg       0.97      0.78      0.84      1115
weighted avg       0.94      0.94      0.93      1115



Of all the selected models, the logistic regression model gave the best results.